In [ ]:
import sagemaker # SageMaker Python SDKcket = sage.Session().default_bucket() # The bucket is where data for Machine Learning is stored
role = sagemaker.get_execution_role() # The role is what gives rights to SageMaker to access resources in the cloud    

In the first part we initialized the [S3 bucket](https://aws.amazon.com/it/s3/) and the [IAM role](https://aws.amazon.com/it/iam/) that will be used by the [SageMaker](https://aws.amazon.com/it/sagemaker/) instance.

In [ ]:
import os # interactions with the operating system, allows you to use the files
import numpy as np # library for mathematical operations, allows to use multidimensional arrays
from sagemaker.rl import RLEstimator, RLToolkit, RLFramework 
from game import TicTacToeGame # This will be changed in order to import @leonardo's game

Then we imported all the libraries we would need, in particular the [SageMaker Python SDK](https://sagemaker.readthedocs.io/en/stable/), which we will use to interact with the SageMaker instance.

In [ ]:
# Set up an estimator object for training a reinforcement learning model
estimator = RLEstimator(
    source_dir="src",  # Directory containing the training code
    entry_point="train-coach.py",  # Entry point file for training
    dependencies=["common/sagemaker_rl"],  # Required dependencies
    toolkit=RLToolkit.COACH,  # Reinforcement learning toolkit to use
    toolkit_version="0.11.0",  # Version of the toolkit
    framework=RLFramework.MXNET,  # Deep learning framework to use
    role=role,  # IAM role with appropriate permissions
    instance_count=1,  # Number of instances for training
    instance_type="ml.m4.xlarge",  # Instance type for training
    output_path=f"s3://{bucket}/",  # S3 output path for model artifacts
    base_job_name="DEMO-rl-tic-tac-toe",  # Base name for the training job
    hyperparameters={"save_model": 1},  # Hyperparameters for training
)

# Start the training process
estimator.fit()

[`RLEstimator`](https://sagemaker.readthedocs.io/en/stable/frameworks/rl/sagemaker.rl.html) is a wrapper of the [Reinforcement Learning](https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-rl-workflow.html) option of SageMaker.

In [ ]:
!pygmentize ./src/deploy-coach.py

In [ ]:
# Deploy the trained model as a predictor
predictor = estimator.deploy(
    initial_instance_count=1,  # Number of initial instances for deployment
    instance_type="ml.m4.xlarge",  # Instance type for deployment
    entry_point="deploy-coach.py"  # Entry point file for deployment
)